In [1]:
from pathlib import Path
import os

if Path.cwd().name != 'src':
    os.chdir('/home/vaschetti/maxarSrc/src')

In [2]:
from my_functions.assemble import names
events_names = names.get_all_events()

In [3]:
for i, name in enumerate(events_names):
    print(i, name)

0 southafrica-flooding22
1 Kalehe-DRC-Flooding-5-8-23
2 shovi-georgia-landslide-8Aug23
3 Emilia-Romagna-Italy-flooding-may23
4 NWT-Canada-Aug-23
5 Libya-Floods-Sept-2023
6 Gambia-flooding-8-11-2022
7 New-Zealand-Flooding23
8 Hurricane-Fiona-9-19-2022
9 Hurricane-Idalia-Florida-Aug23
10 Hurricane-Ian-9-26-2022
11 yellowstone-flooding22
12 pakistan-flooding22
13 Morocco-Earthquake-Sept-2023
14 Indonesia-Earthquake22
15 cyclone-emnati22
16 Kahramanmaras-turkey-earthquake-23
17 Marshall-Fire-21-Update
18 India-Floods-Oct-2023
19 Maui-Hawaii-fires-Aug-23
20 tonga-volcano21
21 BayofBengal-Cyclone-Mocha-May-23
22 Sudan-flooding-8-22-2022


In [4]:
from my_functions.configs import DetectConfig, SegmentConfig
#seg_config = SegmentConfig(batch_size = 1, device='cuda:2', size = 600, stride = 400)
det_config = DetectConfig(batch_size = 1,
                          device='cuda:2',
                          size = 600,
                          stride = 600,
                          TEXT_PROMPT='bush',
                          BOX_THRESHOLD=0.12,
                          min_ratio_GD_boxes_edges=0
                          )

In [5]:
from my_functions.assemble import holders, names

events_names = names.get_all_events()

event = holders.Event(events_names[3],
                      #seg_config = seg_config,
                      det_config = det_config,
                      when='pre')

print(event.region_name)

all_mosaics_names = event.all_mosaics_names
print(all_mosaics_names)

m0 = event.mosaics[all_mosaics_names[-2]]
print(f'Mosaic name: {m0.name}')
print(f'Number of tiles: {m0.tiles_num}')

Europe-Full
['10300100C5C14E00', '1050010032B2A600', '104005003ADC5C00', '10300500CCF8B300', '103005009DF96A00', '1040010030544E00', '1050410012C31100', '10300100BF164000', '10300100C48E0A00', '10300500DDF89E00']
Mosaic name: 10300100C48E0A00
Number of tiles: 15


In [6]:
for i, path in enumerate(m0.tiles_paths):
    print(i, str(path).split('/')[-1])

0 031111231000.tif
1 031111231002.tif
2 031111231021.tif
3 031111212331.tif
4 031111231003.tif
5 031111231001.tif
6 031111213220.tif
7 031111231010.tif
8 031111231012.tif
9 031111230111.tif
10 031111213232.tif
11 031111212333.tif
12 031111231030.tif
13 031111213223.tif
14 031111213222.tif


In [7]:
res = m0.detect_trees_tile(m0.tiles_paths[14])

final text_encoder_type: bert-base-uncased

- GD model device: cuda:2


 96%|█████████▋| 868/900 [03:58<00:08,  3.65it/s]


Discarded empty chips:  32
True num of batch:  868.0


In [8]:
~(res.geometry.area > 6000).all()

True

In [9]:
len(res)

19357

In [10]:
res.to_file("trees_25_3_24_Em_Rom")

In [7]:
res[0]

array([ 1808.9430542 , 16808.05786133,  2200.04272461,  2200.05755615])

In [6]:
box

tensor([[0.6742, 0.5001, 0.6518, 1.0000],
        [0.4174, 0.1213, 0.0641, 0.0977],
        [0.8966, 0.2891, 0.2067, 0.5784],
        [0.6268, 0.2973, 0.3135, 0.5958],
        [0.3973, 0.4996, 0.1009, 0.1841],
        [0.7323, 0.2991, 0.5352, 0.5987],
        [0.4845, 0.0220, 0.0526, 0.0442],
        [0.7265, 0.4994, 0.5475, 1.0000]])

In [11]:
from groundingdino.util.inference import Model
det = Model.post_process_result(source_h = 600,
                          source_w = 600,
                          boxes = box,
                          logits = log)

In [14]:
det.xyxy

array([[ 2.0894305e+02,  5.7861328e-02,  6.0004272e+02,  6.0005756e+02],
       [ 2.3118320e+02,  4.3495026e+01,  2.6965045e+02,  1.0211229e+02],
       [ 4.7595953e+02, -7.7209473e-02,  6.0000555e+02,  3.4698505e+02],
       [ 2.8204865e+02, -3.4008789e-01,  4.7012793e+02,  3.5714130e+02],
       [ 2.0810159e+02,  2.4451244e+02,  2.6861993e+02,  3.5499356e+02],
       [ 2.7880759e+02, -1.3945007e-01,  5.9994080e+02,  3.5909802e+02],
       [ 2.7491299e+02, -7.7271461e-02,  3.0650290e+02,  2.6418896e+01],
       [ 2.7163971e+02, -3.3309937e-01,  6.0012323e+02,  5.9966614e+02]],
      dtype=float32)

In [8]:
phr

['green tree', '', '', '', '', '', '', '']